In [ ]:
# Dependencies
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
import plotly.express as px
import scipy.stats as stats
from scipy.stats import linregress
from urllib.request import urlopen
from scipy import stats
from sodapy import Socrata
from census import Census
from config import appToken, api_key

In [ ]:
# Use open client to obtain CDC COVID-19 death rates, including age/sex data
client = Socrata("data.cdc.gov", appToken)
results = client.get("9bhg-hcku", limit=4000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
# Convert state values for New York City to New York state
results_df['state'] = results_df['state'].replace(['New York City'],'New York')

# Filter DataFrame for State COVID-19 death totals only
results_states = results_df.loc[(results_df['state'] != 'United States') & (results_df['age_group'] == 'All Ages') & (results_df['group'] == 'By Total') & (results_df['sex'] == 'All Sexes')]
results_states = results_states[['state', 'covid_19_deaths']]

# Create copy of DataFrame for tabulation, convert to int, combine New York values, and sort death counts descending
converted_states_df = results_states.copy()
converted_states_df = converted_states_df.astype({"covid_19_deaths": "int64"})
converted_states_df.groupby('state').agg({'covid_19_deaths': 'sum'})
converted_states_df2 = converted_states_df.groupby('state')['covid_19_deaths'].sum()
states_df = pd.DataFrame(converted_states_df2).sort_values('covid_19_deaths',ascending=False)
states_df

In [ ]:
# Plot COVID-19 death totals by State
# Set x axis and tick locations
x_axis = np.arange(len(states_df['covid_19_deaths']))
tick_locations = [value+0.4 for value in x_axis]

# Create a list indicating where to write x labels and set figure size to adjust for space
# Also assign separate color for four states with highest rates
states = list(states_df.index.values)
plt.figure(figsize=(14,6))
barplot = plt.bar(x_axis, states_df['covid_19_deaths'], color='blue', alpha=0.5, align="edge")
plt.xticks(tick_locations, states, rotation="vertical")
barplot[0].set_color('red')
barplot[1].set_color('red')
barplot[2].set_color('red')
barplot[3].set_color('red')

# Set x and y limits
plt.xlim(-0.25, len(x_axis))
plt.ylim(0, max(states_df['covid_19_deaths'])+4000)

# Set Title and Labels
plt.title("Total COVID-19 Deaths by State")
plt.xlabel("State")
plt.ylabel("Deaths")

# Display the chart, and export for later use
plt.tight_layout()
plt.savefig("Images/deathct_by_state.png")
plt.show()

In [ ]:
# Filter CDC data for the four states with highest COVID-19 deaths, to include
# data for both sexes, all ages
results_sex = results_df.loc[((results_df['state'] == 'New York') | (results_df['state'] == 'Texas') | (results_df['state'] == 'California') | (results_df['state'] == 'Florida')) & (results_df['age_group'] == 'All Ages') & (results_df['group'] == 'By Total') & (results_df['sex'] != 'All Sexes')]
results_sex = results_sex[['state', 'sex', 'covid_19_deaths']]
results_sex = results_sex.astype({"covid_19_deaths": "int64"})

# Combine New York state data, convert death total to integer for summation, and reset index
results_sex = results_sex.groupby(['state','sex']).agg({'covid_19_deaths': 'sum'})
converted_sex_df = pd.DataFrame(results_sex)
converted_sex_df.reset_index(drop=False, inplace=True)
converted_sex_df

In [ ]:
# Create lists for plotting deaths by sex
state_lst = []
male_ct = converted_sex_df[converted_sex_df["sex"] == 'Male']["covid_19_deaths"].tolist()
female_ct = converted_sex_df[converted_sex_df["sex"] == 'Female']["covid_19_deaths"].tolist()

for state in converted_sex_df['state']:
    if state in state_lst:
        pass
    else:
        state_lst.append(state)

# Plot COVID-19 death counts by State, stacking bars by Sex, assigning title/legend
plt.bar(state_lst, male_ct, color='b', label="Male", alpha=0.5)
plt.bar(state_lst, female_ct, bottom=male_ct, color='r',label="Female", alpha=0.5)
plt.title("COVID-19 Deaths by Sex")
plt.legend(loc="upper center")

# Display total counts by sex at the top of each bar
for i in range(len(state_lst)):
    plt.text(i, male_ct[i], male_ct[i], ha="center", va="top", color="w", fontweight="bold")
    plt.text(i, female_ct[i]+male_ct[i], female_ct[i], ha="center", va="top", color="w", fontweight="bold")

# Display the chart, and export for later use
plt.savefig("Images/deathct_by_sex.png")
plt.show()

In [ ]:
# Filter CDC data for the three states with highest COVID-19 deaths, to include
# death data for all age groups without overlap
results_age = results_df.loc[((results_df['state'] == 'New York') | (results_df['state'] == 'Texas') | (results_df['state'] == 'California') | (results_df['state'] == 'Florida')) & (results_df['age_group'] != 'All Ages') & (results_df['group'] == 'By Total') & (results_df['sex'] == 'All Sexes')]
results_age2 = results_age.loc[(results_df['age_group'] == '0-17 years') | (results_df['age_group'] == '18-29 years') | (results_df['age_group'] == '30-39 years') | (results_df['age_group'] == '40-49 years') | (results_df['age_group'] == '50-64 years') | (results_df['age_group'] == '65-74 years') | (results_df['age_group'] == '75-84 years') | (results_df['age_group'] == '85 years and over')]
results_age2 = results_age2[['state', 'age_group', 'covid_19_deaths']]

# Create copy of DataFrame for tabulation, combine New York state data, convert death total to integer for summation
age_df = results_age2.copy()
age_df = results_age2.astype({"covid_19_deaths": "int64"})
age_df = age_df.groupby(['state','age_group']).agg({'covid_19_deaths': 'sum'})
converted_age_df = pd.DataFrame(age_df)
converted_age_df.reset_index(drop=False, inplace=True)
converted_age_df

In [ ]:
# Create lists for plotting death by age group by state
age_range = []
cali_ct = age_df[age_df["state"] == 'California']["covid_19_deaths"].tolist()
tex_ct = age_df[age_df["state"] == 'Texas']["covid_19_deaths"].tolist()
ny_ct = age_df[age_df["state"] == 'New York']["covid_19_deaths"].tolist()
flor_ct = age_df[age_df["state"] == 'Florida']["covid_19_deaths"].tolist()

for age in age_df['age_group']:
    if age in age_range:
        pass
    else:
        age_range.append(age)
        
# Set index length and bar width 
index = np.arange(8)
bar_width = 0.20
        
# Plot COVID-19 death counts by State grouped by Age Group
fig, ax = plt.subplots()
cali = ax.bar(index, cali_ct, bar_width, label="California", color="blue", alpha=0.5)
ny = ax.bar(index-bar_width, ny_ct, bar_width, label="New York", color="green", alpha=0.5)
texas = ax.bar(index+bar_width, tex_ct, bar_width, label="Texas", color="red", alpha=0.5)
florida = ax.bar(index+bar_width+bar_width, flor_ct, bar_width, label="Florida", color="orange", alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Age Group')
ax.set_ylabel('Deaths')
ax.set_title('COVID-19 Deaths by Age Group')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(age_range, rotation=45, ha="right")
ax.legend()

# Display the chart, and export for later use
plt.tight_layout()
plt.savefig("Images/deathct_by_agegrp.png")
plt.show()

In [ ]:
# Use open client to obtain CDC race and death data
results2 = client.get("ks3g-spdg", limit=4000000)

# Convert to pandas DataFrame
results2_df = pd.DataFrame.from_records(results2)
results2_df

In [ ]:
# Filter CDC data for the four states with highest COVID-19 deaths, to include
# death data for all race groups excepting nominal "Unknown" figures
results_race = results2_df.loc[((results2_df['state'] == 'New York City') | (results2_df['state'] == 'New York') |(results2_df['state'] == 'Texas') | (results2_df['state'] == 'California') | (results2_df['state'] == 'Florida')) & (results2_df['race_and_hispanic_origin'] != 'Total Deaths') & (results2_df['race_and_hispanic_origin'] != 'Unknown')]
results_race = results_race.loc[(results_race['age_group_new'] == '0-17 years') | (results_race['age_group_new'] == '18-29 years') | (results_race['age_group_new'] == '30-39 years') | (results_race['age_group_new'] == '40-49 years') | (results_race['age_group_new'] == '50-64 years') | (results_race['age_group_new'] == '65-74 years') | (results_race['age_group_new'] == '75-84 years') | (results_race['age_group_new'] == '85 years and over')]
results_race2 = results_race.fillna(0)
results_race2 = results_race2[['state', 'race_and_hispanic_origin', 'covid_19_deaths']]

results_race2['state'] = results_race2['state'].replace(['New York City'],'New York')

# Create copy of DataFrame for tabulation, convert death total to integer for summation
converted_race_df = results_race2.copy()
converted_race_df = converted_race_df.astype({"covid_19_deaths": "int64"})

# Group DataFrame by State, Race cagtegories, combining data for New York
group_race_df = converted_race_df.groupby(['state','race_and_hispanic_origin']).agg({'covid_19_deaths': 'sum'})
race_df = pd.DataFrame(group_race_df["covid_19_deaths"])

# Convert DataFrame to pivot table
dfr = race_df.pivot_table(index='race_and_hispanic_origin', columns='state', values='covid_19_deaths').sort_values('Texas',ascending=False)
dfr

In [ ]:
# Create lists for plotting deaths by race
race_range = dfr.index.tolist()
cali_ct2 = dfr["California"].tolist()
ny_ct2 = dfr["New York"].tolist()
tex_ct2 = dfr["Texas"].tolist()
flor_ct2 = dfr["Florida"].tolist()

# Set index length and bar width 
index = np.arange(7)
bar_width = 0.2
        
# Plot COVID-19 death counts for each State by Race 
fig, ax = plt.subplots()
cali2 = ax.barh(index+bar_width+bar_width, cali_ct2, bar_width, label="California", color="blue", alpha=0.5)
ny2 = ax.barh(index+bar_width, ny_ct2, bar_width, label="New York", color="green", alpha=0.5)
texas2 = ax.barh(index-bar_width, tex_ct2, bar_width, label="Texas", color="red", alpha=0.5)
florida2 = ax.barh(index, flor_ct2, bar_width, label="Florida", color="orange", alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Deaths')
ax.set_title('COVID-19 Deaths by Race and Hispanic Origin')
ax.set_yticks(index + bar_width / 2)
ax.set_yticklabels(race_range)
ax.legend(loc="upper right")

# Display the chart, and export for later use
plt.savefig("Images/deathct_by_racegrp.png", bbox_inches='tight')
plt.figure(figsize=(30,10))
plt.tight_layout()
plt.show()

In [ ]:
# Import Census api key
from config import api_key

# Get 2020 Census data by race for the four states under study
census_2020_url = "https://api.census.gov/data/2020/dec/pl?get=NAME,P2_005N,P2_002N,P2_006N,P2_008N,P2_007N,P2_011N,P2_009N&for=state:06,12,36,48&key=" + api_key
census_data = requests.get(census_2020_url).json()

In [ ]:
# Create DataFrame of census data and create header row
census_pd = pd.DataFrame(census_data).T
new_header = census_pd.iloc[0]
census_pd = census_pd[1:8]
census_pd.columns = new_header 

# Create lists of death counts by state and add to DataFrame
cali_data = census_pd['California'].astype(int).tolist()
ny_data = census_pd['New York'].astype(int).tolist()
flor_data = census_pd['Florida'].astype(int).tolist()
tex_data = census_pd['Texas'].astype(int).tolist()

dfr['California Total Pop.'] = cali_data
dfr['New York Total Pop.'] = ny_data
dfr['Florida Total Pop.'] = flor_data
dfr['Texas Total Pop.'] = tex_data

# Sum total deaths and population
sum_cali_deaths = sum(dfr['California'])
sum_ny_deaths = sum(dfr['New York'])
sum_flor_deaths = sum(dfr['Florida'])
sum_tex_deaths = sum(dfr['Texas'])

sum_cali = sum(dfr['California Total Pop.'])
sum_ny = sum(dfr['New York Total Pop.'])
sum_flor = sum(dfr['Florida Total Pop.'])
sum_tex = sum(dfr['Texas Total Pop.'])

# Create DataFrame summarizing populations and death rates by state with mapping and date stamp
death_rates = pd.DataFrame({
    "State" : ["California", "Florida", "New York", "Texas"],
    "Population" : [sum_cali, sum_flor, sum_ny, sum_tex],
    "COVID-19 Deaths" : [sum_cali_deaths, sum_flor_deaths, sum_ny_deaths, sum_tex_deaths],
    "Death Rate %" : [sum_cali_deaths / sum_cali * 100,
                      sum_flor_deaths / sum_flor * 100,
                      sum_ny_deaths / sum_ny * 100,
                      sum_tex_deaths / sum_tex * 100,],
})

death_rates["Population"] = death_rates["Population"].map("{:,.0f}".format)
death_rates["COVID-19 Deaths"] = death_rates["COVID-19 Deaths"].map("{:,.0f}".format)
death_rates["Death Rate %"] = death_rates["Death Rate %"].map("{:.2f}%".format)

today = pd.to_datetime("today")

death_rates
death_rates.style.set_caption(f"Update: {today:%m/%d/%Y}")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; color:black;'
 }], overwrite=False)

In [ ]:
# Display working data for chi-square testing
dfr

In [ ]:
# Calculate death count % and population % by race for chi-squared test and show summary table with date stamp
dfr["CA Deaths %"] = dfr['California']/sum_cali_deaths*100
dfr["CA Population %"] = dfr['California Total Pop.']/sum_cali*100


dfr["FL Deaths %"] = dfr['Florida']/sum_flor_deaths*100
dfr["FL Population %"] = dfr['Florida Total Pop.']/sum_flor*100


dfr["NY Deaths %"] = dfr['New York']/sum_ny_deaths*100
dfr["NY Population %"] = dfr['New York Total Pop.']/sum_ny*100

dfr["TX Deaths %"] = dfr['Texas']/sum_tex_deaths*100
dfr["TX Population %"] = dfr['Texas Total Pop.']/sum_tex*100


dfr = dfr[["CA Deaths %", "CA Population %", "FL Deaths %", "FL Population %", "NY Deaths %", "NY Population %", "TX Deaths %", "TX Population %"]]

dfr_disp = dfr.copy()

dfr_disp["CA Deaths %"] = dfr_disp["CA Deaths %"].map("{:.2f}".format)
dfr_disp["CA Population %"] = dfr_disp["CA Population %"].map("{:.2f}".format)
dfr_disp["FL Deaths %"] = dfr_disp["FL Deaths %"].map("{:.2f}".format)
dfr_disp["FL Population %"] = dfr_disp["FL Population %"].map("{:.2f}".format)
dfr_disp["NY Deaths %"] = dfr_disp["NY Deaths %"].map("{:.2f}".format)
dfr_disp["NY Population %"] = dfr_disp["NY Population %"].map("{:.2f}".format)
dfr_disp["TX Deaths %"] = dfr_disp["TX Deaths %"].map("{:.2f}".format)
dfr_disp["TX Population %"] = dfr_disp["TX Population %"].map("{:.2f}".format)

today = pd.to_datetime("today")

dfr_disp
dfr_disp.index.name=''
dfr_disp.columns.name=''
dfr_disp.style.set_caption(f"Update: {today:%m/%d/%Y}")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; color:black;'
 }], overwrite=False)

In [ ]:
# Hypothesis: Demographic particulars such as sex, age, race, et al., may impact COVID-19 death outcomes.
# Null Hypothesis: Demographic particulars do not impact COVID-19 death outcomes.

# Race Test: Conduct chi-square test to see if death counts for any states show significance compared to
# general population by race group.

# The degree of freedom is 7-1 = 6 (two columns only for each state)
# With a p-value of 0.05, the confidence level is 1.00-0.05 = 0.95.
critical_value = stats.chi2.ppf(q = 0.95, df = 6)
# The critical value
critical_value

In [ ]:
stats.chisquare(dfr['CA Deaths %'], dfr['CA Population %']) # California test

In [ ]:
stats.chisquare(dfr['FL Deaths %'], dfr['FL Population %']) # Florida test

In [ ]:
stats.chisquare(dfr['NY Deaths %'], dfr['NY Population %']) # New York test

In [ ]:
stats.chisquare(dfr['TX Deaths %'], dfr['TX Population %']) # Texas test

In [ ]:
# Date stamp for test result
today = pd.to_datetime("today")
print(f"Update: {today:%m/%d/%Y}")

### Conclusion
* Since the chi-square values at a confidence level of 95% are much lower than the critical value of 12.6, we conclude that the differences seen across race categories compared to the states' general populations are not statistically significant.

In [ ]:
# Use open client to obtain data
client = Socrata("data.cdc.gov", appToken)
results = client.get("dnhi-s2bf", limit = 1000000)

# Convert to pandas DataFrame
death_quarterly = pd.DataFrame.from_records(results)

In [ ]:
# Cleaning columns and dropping NA values
death_quarterly_1 = death_quarterly.drop(columns=['footnote'])
death_quarterly_2 = death_quarterly_1.dropna(subset =['covid_19_deaths','total_deaths'])

In [ ]:
# Setting numerical values to int for future calculations
death_quarterly_2['total_deaths'] = death_quarterly_2['total_deaths'].astype(int)
death_quarterly_2['covid_19_deaths'] = death_quarterly_2['covid_19_deaths'].astype(int)

#Group by county level
death_gb = death_quarterly_2.groupby(['fips_code'])
death_gb_df = death_gb.sum()

In [ ]:
# Calculating new column to calculate covid death %
death_gb_df['covid_portion_of_total_deaths'] = death_gb_df['covid_19_deaths']/death_gb_df['total_deaths']
death_gb_df = pd.DataFrame(death_gb_df)
death_gb_df.reset_index(inplace=True)

In [ ]:
# Generating stats on the datafame to get range for plot
death_gb_df.describe(include='all')

In [ ]:
# URL for GET requests to retrieve county data
url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
response = requests.get(url).json()

In [ ]:
# Creating map of polygons representing county boundaries
fig = px.choropleth(death_gb_df, geojson=response, locations='fips_code', color='covid_portion_of_total_deaths',
                           color_continuous_scale="ylgnbu",
                           range_color=(0, .35),
                           scope="usa",
                           labels={'covid_portion_of_total_deaths':'Covid Deaths as % of Total Deaths'}
                          )
fig.show()

In [ ]:
# Renaming column for later merge
death_df = death_gb_df.rename(columns = {'fips_code':'fips'})

In [ ]:
# Use open client to obtain data - can take up to 5 minutes to get results in dataframe ~1.6M rows
client = Socrata("data.cdc.gov", appToken)
results = client.get("8xkx-amqh", limit = 2000000)
                     
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
# Limiting to specific columns
results_df_columns = results_df[['date','fips','recip_state','recip_county',
                                     'completeness_pct','administered_dose1_recip',
                                     'administered_dose1_pop_pct','series_complete_yes',
                                     'series_complete_pop_pct','metro_status','census2019']]

In [ ]:
# Setting up a clean datafame with out any NA and setting a date column
clean_df = results_df_columns.dropna()
clean_df['date'] = pd.to_datetime(clean_df['date']).dt.date

In [ ]:
# Inserting Year Quarter Month columns
clean_df['month'] = pd.DatetimeIndex(clean_df['date']).month
clean_df['quarter'] = pd.DatetimeIndex(clean_df['date']).quarter
clean_df['year'] = pd.DatetimeIndex(clean_df['date']).year

In [ ]:
# Grouping by county and taking max value
grouped_fips = clean_df.groupby(['fips'])
max_group = grouped_fips.max()

In [ ]:
# Creating a data frame by vaccine dose
max_dose = pd.DataFrame(max_group[['administered_dose1_recip','administered_dose1_pop_pct']]) 

In [ ]:
# Merging Death dataframe and Vaccine data frame
merged_df = death_df.merge(max_dose,on='fips')

In [ ]:
# Renaming columns from deaths dataframe and assigning proper types to columns
death_quarterly_2 = death_quarterly.rename(columns = {'fips_code' : 'fips'})
death_quarterly_2['year']= death_quarterly_2['year'].astype(int)
death_quarterly_2['quarter']=death_quarterly_2['quarter'].astype(int)
clean_df['year']=clean_df['year'].astype(int)
clean_df['quarter']=clean_df['quarter'].astype(int)

In [ ]:
# Merging deaths and our clean dataframe and limiting to 4 states CA,FL,TX,NY
total_quarter = pd.merge(death_quarterly_2,clean_df, on = ('fips','year','quarter'),how ='inner')
total_quarter_2 = total_quarter.loc[(total_quarter['state_of_residence']=='Florida')|(total_quarter['state_of_residence'] =='Texas')|(total_quarter['state_of_residence'] =='California')|(total_quarter['state_of_residence'] =='New York'),:]

In [ ]:
# Inserting quarter column for plotting purposes
total_quarter_2['quarter_number'] = pd.to_datetime(total_quarter_2['year'].astype(str) + 'Q' + total_quarter_2['quarter'].astype(str))

In [ ]:
# Cleaning out any NA from merge and calculating covid death rate
total_quarter_2.dropna(subset = ['covid_19_deaths','total_deaths'])
total_quarter_2['covid_death_rate']= total_quarter_2['covid_19_deaths'].astype(float)/total_quarter_2['total_deaths'].astype(float)

In [ ]:
# Removing duplicate values
total_quarter_2.drop_duplicates(subset =['quarter_number','fips'],inplace=True)                                                                                                                                    

In [ ]:
# Column names to use for plots
total_quarter_2.columns

In [ ]:
# Setting proper variable types
total_quarter_2['administered_dose1_recip'] = total_quarter_2['administered_dose1_recip'].astype(float) 
total_quarter_2['census2019'] = total_quarter_2['census2019'].astype(float) 
total_quarter_2['covid_19_deaths'] = total_quarter_2['covid_19_deaths'].astype(float) 
total_quarter_2['total_deaths'] = total_quarter_2['total_deaths'].astype(float) 

In [ ]:
# Creating df for California
california_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'California',:]
cali_quarter_group = california_df.groupby('quarter_number')
cali_quarter_df = cali_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
cali_quarter_df['covid_death_rate']= cali_quarter_df['covid_19_deaths']/cali_quarter_df['total_deaths']
cali_quarter_df['vaccine_rate']= cali_quarter_df['administered_dose1_recip']/cali_quarter_df['census2019']


In [ ]:
# Setting proper variables for original quarter dataframe
total_quarter['administered_dose1_recip'] = total_quarter['administered_dose1_recip'].astype(float) 
total_quarter['census2019'] = total_quarter['census2019'].astype(float) 
total_quarter['covid_19_deaths'] = total_quarter['covid_19_deaths'].astype(float) 
total_quarter['total_deaths'] = total_quarter['total_deaths'].astype(float) 

In [ ]:
# Creating df for Florida
florida_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Florida',:]
FL_quarter_group = florida_df.groupby('quarter_number')
FL_quarter_df = FL_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
FL_quarter_df['covid_death_rate']= FL_quarter_df['covid_19_deaths']/FL_quarter_df['total_deaths']
FL_quarter_df['vaccine_rate']= FL_quarter_df['administered_dose1_recip']/FL_quarter_df['census2019']

In [ ]:
# Creating df for Texas
texas_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Texas',:]
TX_quarter_group = texas_df.groupby('quarter_number')
TX_quarter_df = TX_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
TX_quarter_df['covid_death_rate']= TX_quarter_df['covid_19_deaths']/TX_quarter_df['total_deaths']
TX_quarter_df['vaccine_rate']= TX_quarter_df['administered_dose1_recip']/TX_quarter_df['census2019']


In [ ]:
# Creating dataframe for scatter plot
scatter_quarter = total_quarter
scatter_quarter['quarter_number'] = pd.to_datetime(total_quarter['year'].astype(str) + 'Q' + total_quarter['quarter'].astype(str))

In [ ]:
# Limiting to specific columns
scatter_plot = scatter_quarter.loc[:,['fips','administered_dose1_recip','census2019','covid_19_deaths','total_deaths','quarter_number']]

In [ ]:
# Removing duplicates
scatter_plot.drop_duplicates(subset =['quarter_number','fips'],inplace=True)

In [ ]:
# Removing NA values
scatter_plot.dropna()

In [ ]:
# Setting a clean df for the scatter
clean_scatter = scatter_plot.loc[(scatter_plot['administered_dose1_recip'] != 0) & (scatter_plot['covid_19_deaths'] != 0) & (scatter_plot['census2019'] != 0) &(scatter_plot['total_deaths'] != 0),:]

In [ ]:
# Removing 0 values and inf values to get proper rate calculations
clean_scatter.replace([np.inf, -np.inf], np.nan, inplace=True)
clean_scatter.dropna(inplace=True)

In [ ]:
# Scatter plot to show covid death % and vaccine pop fraction
x_x = clean_scatter['administered_dose1_recip']/clean_scatter['census2019']
y_y = clean_scatter['covid_19_deaths']/clean_scatter['total_deaths']

fig1, ax1 = plt.subplots()
plt.scatter(x_x,y_y)
plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')
plt.show()

In [ ]:
# Linear regression on scatter plot
correlation = stats.pearsonr(x_x,y_y)
print(f"The correlation between vaccine percent \033[34mand\033[0m the covid death percent \033[34mis\033[0m \033[31m{round(correlation[0],2)}\033[0m")

# Creating Plot for scatter and adding line
(slope, intercept,rvalue, pvalue, stderr) = linregress(x_x,y_y)
regress = x_x* slope + intercept
line_eq = f"y = {round(slope, 2)} x + {round(intercept, 2)}"

plt.scatter(x_x,y_y, s = 3)
plt.plot(x_x, regress, color='red')
plt.annotate(line_eq,(.4,.5),fontsize=12,color="red")
plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')

# Uncomment below to save image to desired location
#plt.savefig("../Images/Scatter_Vaccine_vs_Death")

plt.show()


In [ ]:
# Creating 2D heatmap from scatter to show concentration
x_min = np.min(x_x)
x_max = np.max(x_x)
  
y_min = np.min(y_y)
y_max = np.max(y_y)

x_bins = np.linspace(x_min, x_max, 70)
y_bins = np.linspace(y_min, y_max, 50)

fig, ax = plt.subplots()
plt.hist2d(x_x, y_y, bins = [x_bins,y_bins],cmap = plt.cm.nipy_spectral)

plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')

# Uncomment below to save image to desired location
#plt.savefig("../Images/Scatter_Vaccine_vs_Death_Heat")

plt.show()

In [ ]:
# Creating df for New York
NY_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'New York',:]
NY_quarter_group = NY_df.groupby('quarter_number')
NY_quarter_df = NY_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
NY_quarter_df['covid_death_rate']= NY_quarter_df['covid_19_deaths']/NY_quarter_df['total_deaths']
NY_quarter_df['vaccine_rate']= NY_quarter_df['administered_dose1_recip']/NY_quarter_df['census2019']

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20

# Plot COVID-19 death & Vaccine counts by State
fig, ax = plt.subplots()
ny = ax.bar(index,NY_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,NY_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('New York COVID-19 Death Rate vs. Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()

# Display the chart, and export for later use
plt.tight_layout()

# Uncomment below to save image to desired location
#plt.savefig('../Images/NY_Covid_Death_vs_Vaccine_Rate.png')

plt.show()

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20

# Plot COVID-19 death & Vaccine counts by State
fig, ax = plt.subplots()
ny = ax.bar(index,cali_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,cali_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('California COVID-19 Death Rate vs. Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()

# Display the chart, and export for later use
plt.tight_layout()

# Uncomment below to save image to desired location
#plt.savefig('../Images/CA_Covid_Death_vs_Vaccine_Rate.png')

plt.show()

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20

# Plot COVID-19 death & Vaccine counts by State
fig, ax = plt.subplots()
ny = ax.bar(index,FL_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,FL_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('Florida COVID-19 Death Rate vs. Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()

# Display the chart, and export for later use
plt.tight_layout()

# Uncomment below to save image to desired location
#plt.savefig('../Images/FL_Covid_Death_vs_Vaccine_Rate.png')

plt.show()